# Decorators

## Simple example of what a decorator does

### Functions are first class objects - what does this mean?

In [10]:
# functions in python are first class objects, and so can be passed as arguments and returned from functions
def log_output():
    print("Logging something")
    

# this is assigning a function to a variable, not executing the function and storing the result
log = log_output

# this executest the function
log()

Logging something


In [11]:
# we can pass the function as a parameter to another function

def log_something(func):
    print("Before...")
    func() # invoke the function passed in as an argument
    print("After...")

# log is a function here
log_something(log)  

Before...
Logging something
After...


### Functions can be defined within functions, and returned - these are known as closures

In [ ]:
# we can now return a function from a function, this is called a closure

def make_case(case_type):
    
    def upper(value):
        return value.upper()

    def lower(value):
        return value.lower()
    

    if case_type == 'u':
        return upper
    else:
        return lower

In [33]:
# If we pass 'u' as the case_type parameter, the upper() closure is returned, otherwise the lower() closure is returned
func = make_case('u')
print(func('Hello'))

func = make_case('l')
print(func('Hello'))

HELLO
hello


In [34]:
# Now if we take this one step further and pass in a function, and return a closure which executes the function
# We can change the way the function passed in as an argument operates

def make_upper(func):
    
    def wrapper():
        # execute the function passed in as an argument
        result = func()
        
        # convert to upper case and return the result
        modified_result =  result.upper()
        return modified_result
    
    # return the modified function
    return wrapper


# this is a base function we want to modify
def greet():
    return "Hello there, how are you?"


greet()

# here we create modify the behaviour of greet() to force to upper case
greet = make_upper(greet)

# eecuting the function now, is exeuting the wrapper function around the original greet function
greet()


'HELLO THERE, HOW ARE YOU?'

## Decorator Syntax

In [35]:
# now we understand how a function can effetively return a function (a closure()), and we can pass in functions
# ...and therefore wrap behaviour around a function to alter its behaviour...we can introduce the decorator syntax

# the syntax demonstrated above, can be simplified using the @ decorator syntax
# this example is effectively exactly the same as: say_goodnight =  make_upper(say_goodnight)
@make_upper
def say_goodnight():
    return "Goodnight everyone"

say_goodnight()

'GOODNIGHT EVERYONE'

In [37]:
# we can now add @make_upper to any function which returns a string value, to alter its behaviour 
@make_upper
def say_something():
    return "Something!!"

say_something()

'SOMETHING!!'

### Applying multiple decorators

In [40]:
# multiple decorators can be applied to a function

# reverse a string
def reverse(func):
    
    def wrapper():
        result = func()
        result = result[::-1]
        return result
    
    return wrapper

# remove first character of a string
def remove_first(func):
    def wrapper():
        result = func()
        result = result[1:]
        return result
    
    return wrapper


# here we add multiple decorators, note how they are executed "upwards"
# i.e. the first character is removed and then the string is reversed
# this is the same as the syntax : process_string = reverse(remove_first(process_string))
@reverse
@remove_first
def process_string():
    return "Hello there!"

process_string()

'!ereht olle'

In [41]:
# here we add multiple decorators, note how they are executed "upwards"
# i.e. the string is reversed and THEN the first character is removed
# this is the same as the syntax : process_string = remove_first(reverse(process_string))
@remove_first
@reverse
def process_string():
    return "Hello there!"

process_string()

'ereht olleH'

In [43]:
# And of course we apply one, both or neither of the decorators
@remove_first
def process_string():
    return "Hello there!"

process_string()

'ello there!'

## Decorators with arguments

In [45]:
# the examples so far have not had to deal with decorating functions with arguments

def reverse(func):
    
    def wrapper():
        result = func()
        result = result[::-1]
        return result
    
    return wrapper


@reverse
def process_string(string_to_process):
    return string_to_process.upper();

In [47]:
# this wont work
try:
    process_string('Hello')
except Exception as e:
    print("Error: ", e)

Error:  wrapper() takes 0 positional arguments but 1 was given


In [51]:
# we solve this with *args and **kwargs
def reverse(func):
    
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        result = result[::-1]
        return result
    
    return wrapper

@reverse
def process_string(string_to_process):
    return string_to_process.upper();

print(process_string('Hello'))
print(process_string(string_to_process = 'Hello'))

OLLEH
OLLEH


## Trace example    

In [64]:
# this example allows us to trace each function call by printing out the function name being called, 
# any arguments and the return value
def trace(func):

    def wrapper(*args, **kwargs):
        print(f"TRACE: calling {func.__name__}() " 
              f"with args {args}, {kwargs}")
        original_result = func(*args, **kwargs)
        print(f"TRACE: {func.__name__}() " 
              f"returned {original_result!r}")
        return original_result
    
    return wrapper


@trace
def program_to_trace(value):
    return value[::-1].upper()

# all the arguments are positional and not keyword arguments
program_to_trace("I like you")
program_to_trace("What is your name?")

@trace
def another_program_to_trace(value):
    return value[::-1].lower()

# note how the trace picks up the keyword arguments this time
another_program_to_trace(value="I like you")
another_program_to_trace(value="What is your name?")

TRACE: calling program_to_trace() with args ('I like you',), {}
TRACE: program_to_trace() returned 'UOY EKIL I'
TRACE: calling program_to_trace() with args ('What is your name?',), {}
TRACE: program_to_trace() returned '?EMAN RUOY SI TAHW'
TRACE: calling another_program_to_trace() with args (), {'value': 'I like you'}
TRACE: another_program_to_trace() returned 'uoy ekil i'
TRACE: calling another_program_to_trace() with args (), {'value': 'What is your name?'}
TRACE: another_program_to_trace() returned '?eman ruoy si tahw'


'?eman ruoy si tahw'